In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 150)
import sqlite3
import glob
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#importing all the files.
csv_files = glob.glob('../data/2020/*.csv')

#Creating the empty dataframes so that the files can be added to.

citibike = pd.DataFrame()


# Iterating over the 2020 files so that they are in a dataframe.
for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory = False)
    citibike = pd.concat([citibike, df])


In [ ]:
#removing spaces out of column names.
citibike = citibike.rename(columns = {'starttime': 'started_at', 'stoptime' : 'ended_at','start station id': 'start_station_id', 'start station name': 'start_station_name', 'start station latitude':'start_lat', 'start station longitude':'start_lng', 'end station id':'end_station_id', 'end station name':'end_station_name', 'end station latitude':'end_lat','end station longitude':'end_lng', 'usertype':'user_type', 'tripduration' : 'trip_duration'})

#removing columns that are not also available in 2021 df
citibike = citibike.drop(columns = ['birth year', 'gender', 'bikeid'])

In [ ]:
#A function that changed the dtype of these columns to datetime.

def convert_datetime(df, columns):
    for column in columns:
        df[column] = pd.to_datetime(df[column])

columns = ['started_at', 'ended_at']

convert_datetime(citibike, columns)

In [ ]:
# A function that will make new columns for the date, time, hour and day of the week each trip began and ended.

def datetime_date_time(df, columns):
    for column in columns:
        df[f'{column}_date'] = pd.to_datetime(df[column]).dt.date
        df[f'{column}_time'] = pd.to_datetime(df[column]).dt.time
        df[f'{column}_hour'] = pd.to_datetime(df[column]).dt.hour
        df[f'{column}_day_of_week'] = pd.to_datetime(df[column]).dt.weekday
        df[f'{column}_day_of_year']= pd.to_datetime(df[column]).dt.dayofyear
        
columns = ['started_at', 'ended_at']

datetime_date_time(citibike, columns)

In [ ]:
#using the convert_datetime function to convert the date column to a datetime dtype.

columns = ['started_at_date', 'ended_at_date']

convert_datetime(citibike, columns)

In [ ]:
#A function that will create a column for the month of the trip.

def datetime_month(df, columns):
    for column in columns:
        df[f'{column}_month'] = pd.DatetimeIndex(df[column]).month
        
columns = ['started_at', 'ended_at']

datetime_month(citibike, columns)

In [ ]:
#https://stackoverflow.com/questions/29688899/pandas-checking-if-a-date-is-a-holiday-and-assigning-boolean-value

#A datframe that will define if a date was a holiday.

dr = pd.date_range(start='2020-01-01', end='2020-12-31')
df = pd.DataFrame()
df['date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

df['holiday'] = df['date'].isin(holidays)


In [ ]:
#2020 
citibike['trip_duration'].describe()

**Using the weather**

In [ ]:
#importing the weather file.
cols = ['PRCP', 'SNOW', 'TMAX', 'TMIN', 'DATE', 'RHAV', 'AWND', 'RHMN', 'RHMX']
weather = pd.read_csv('../data/USW00094728.csv', usecols = cols, low_memory = False)

In [ ]:
#changing all the column titles to lowercase.
weather.columns = map(str.lower, weather.columns)

#changing the DATE column to a datetime datatype.
weather['date'] = pd.to_datetime(weather['date'])

#choosing the years I want data in.
weather = weather[weather['date'].dt.year.isin([2020])].reset_index(drop = True)

In [ ]:
day_trips = citibike.groupby('started_at_date').size().reset_index(name = 'num_of_trips')
day_trips = day_trips.rename(columns = {'started_at_date' : 'date'})

In [ ]:
day_trips = day_trips.merge(df, left_on = 'date', right_on = 'date', how = 'left')
day_trips = day_trips.merge(weather, left_on = 'date', right_on = 'date', how = 'left')

In [ ]:
#Changing the temperature to fahrenheit.
#the original temperature was the celsius temp x 10. 
def to_fahrenheit(df, columns):
    for column in columns:
        df[f'{column}_fahrenheit'] = (df[column]/10)*(9/5)+32
columns = ['tmax', 'tmin']

to_fahrenheit(day_trips, columns)

In [ ]:
#Droping the original temp columns and renaming the farenheit temp columns.

day_trips = day_trips.drop(columns = ['tmax','tmin'])
day_trips = day_trips.rename(columns = {'tmax_fahrenheit' : 'tmax', 'tmin_fahrenheit' : 'tmin'})

day_trips['tavg'] = (day_trips['tmax'] + day_trips['tmin'])/2

In [ ]:
# day_trips['tavg'] = (day_trips['tmax'] + day_trips['tmin'])/2

In [ ]:
median_trips = round(citibike.groupby('started_at_date')['trip_duration'].median(), 2) #df that has the date and medians trip duration.
day_trips = day_trips.merge(median_trips, left_on = 'date', right_on = 'started_at_date', how = 'left') #merging the df to day_trips.
day_trips = day_trips.rename(columns = {'trip_duration' : 'median_trip_duration'}) #changing the name of the column.

In [ ]:
mean_trips = round(citibike.groupby('started_at_date')['trip_duration'].mean(), 2)
day_trips = day_trips.merge(mean_trips, left_on = 'date', right_on = 'started_at_date', how = 'left') #merging the df to day_trips.
day_trips = day_trips.rename(columns = {'trip_duration' : 'mean_trip_duration'}) #changing the name of the column.

In [ ]:
#Adding a day of the week column.
day_trips['day_of_week'] = pd.to_datetime(day_trips['date']).dt.weekday

In [ ]:
day_trips

In [ ]:
# #saving df to csv
# day_trips.to_csv('../data/day_trips_2020')

***Answering data questions***

*What time are the bikes most frequently used during the day?*

In [ ]:
#The hour the most amount of trips occur.
def most_hour (df, year):
    hour_count = df.groupby('started_at_hour')['started_at_hour'].count().sort_values(ascending=False)
    most_popular = hour_count.index[0]
    number_of_rides = hour_count.iloc[0]
    print(f"The hour with the most amount of trips in {year} was {most_popular} with {number_of_rides} rides.")

most_hour(citibike, 2020)

*What time are the bikes used the least during the day?*

In [ ]:
#The hour the least amount of trips occur.
def least_hour (df, year):
    hour_count = df.groupby('started_at_hour')['started_at_hour'].count().sort_values(ascending=True)
    least_popular = hour_count.index[0]
    number_of_rides = hour_count.iloc[0]
    print(f"The hour with the least amount of trips in {year} was {least_popular} with {number_of_rides} rides.")

least_hour(citibike, 2020)

*Which stations are the most frequent for starting & ending a trip?*

In [ ]:
#The station where the most trips begin
def popular_starting_station (df, year):
    station_count = df.groupby('start_station_name')['start_station_name'].count().sort_values(ascending=False)
    most_popular = station_count.index[0]
    number_of_rides = station_count.iloc[0]
    print(f"The station where most trips began in {year} was {most_popular} with {number_of_rides} rides.")

popular_starting_station(citibike, 2020)

In [ ]:
#The station where the most trips ended.
def popular_ending_station (df, year):
    station_count = df.groupby('end_station_name')['end_station_name'].count().sort_values(ascending=False)
    most_popular = station_count.index[0]
    number_of_rides = station_count.iloc[0]
    print(f"The station where most trips ended in {year} was {most_popular} with {number_of_rides} rides.")

popular_ending_station(citibike, 2020)

**What is the average trip length? Does it change depending on day or time of day?**

In [ ]:
#The average bike trip.
def avg_bike_trip (df, year):
    for column in columns:
        avg_trip_len = round(df[column].mean())
        print(f"The average length of a bike trip in {year} is {avg_trip_len} seconds")
        
columns = ['trip_duration']
avg_bike_trip(citibike, 2020)

In [ ]:
#The hour with the longest bike trip.
def avg_bike_trip_by_hour (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_hour')['trip_duration'].mean().sort_values(ascending = False))
        hour_with_longest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the hour that has the longest bike trip on average is hour {hour_with_longest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_hour(citibike, 2020)

In [ ]:
#The hour with the shortest bike trip.
def avg_bike_trip_by_hour_least (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_hour')['trip_duration'].mean().sort_values(ascending = True))
        hour_with_shortest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the hour that has the shortest bike trip on average is hour {hour_with_shortest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_hour_least(citibike, 2020)

In [ ]:
#The day of year with the longest trips.
def avg_bike_trip_by_day_of_year (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_day_of_year')['trip_duration'].mean().sort_values(ascending = False))
        day_of_year_with_longest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the day of year that has the longest bike trip on average is day {day_of_year_with_longest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_day_of_year(citibike, 2020)

In [ ]:
#The day of year with the shortest trips.
def avg_bike_trip_by_day_of_year_shortest (df, year):
    for column in columns:
        avg_trip_len = round(df.groupby('started_at_day_of_year')['trip_duration'].mean().sort_values(ascending = True))
        day_of_year_with_shortest_trips = avg_trip_len.index[0]
        avg_seconds = avg_trip_len.iloc[0]
        print(f"In {year}, the day of year that has the shortest bike trip on average is day {day_of_year_with_shortest_trips}, which lasts about {avg_seconds} seconds")
        
avg_bike_trip_by_day_of_year_shortest(citibike, 2020)

In [ ]:
citibike

In [ ]:
weather

In [ ]:
weather['wsf2'].value_counts()